## Creando el modelo predictivo

In [1]:
import numpy as np
import cv2
import tensorflow as tf

In [2]:
from scipy.special import softmax

In [3]:
# Importar el modelo de deteccion de rostros
face_detection_model = cv2.dnn.readNetFromCaffe('./Models/deploy.prototxt.txt',
                                                './Models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

# Importar el modelo de reconocimiento de mascarillas
model = tf.keras.models.load_model('modelo_cnn_deteccion_mascarillas_AG_MT_V1')

In [4]:
# Crear las etiquetas y asignarles un color respectivo a cada una de ellas
etiquetas = ['Con Mascarilla', 'Sin Mascarilla']

def getcolor(etiqueta):
    if etiqueta == "Con Mascarilla":
        color = (0,255,0)
    else: 
        color = (0,0,255)
    return color

In [5]:
getcolor('Sin Mascarilla')

(0, 0, 255)

In [6]:
#####FORMA NORMAL#####
img = cv2.imread('./mascarilla2.jpg')
# Paso 1 - Deteccion de rostros
image = img.copy()
h, w = image.shape[:2]
blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,117,123),swapRB=True)
#
face_detection_model.setInput(blob)
detection = face_detection_model.forward()
for i in range(0,detection.shape[2]):
    confidence = detection[0,0,i,2]
    if confidence > 0.5:
        box = detection[0,0,i,3:7]*np.array([w,h,w,h])
        box = box.astype(int)
        pt1 = (box[0],box[1])
        pt2 = (box[2],box[3])
        cv2.rectangle(image,pt1,pt2,(0,255,0),1)    
        # Paso 2 - Preprosesamiento de los datos
        face = image[box[1]:box[3],box[0]:box[2]]
        face_blob = cv2.dnn.blobFromImage(face,1,(100,100),(104,117,123),swapRB=True)
        face_blob_squeeze = np.squeeze(face_blob).T
        face_blob_rotate = cv2.rotate(face_blob_squeeze,cv2.ROTATE_90_CLOCKWISE)
        face_blob_flip = cv2.flip(face_blob_rotate,1)
        # Paso 3 - Normalizacion
        img_norm = np.maximum(face_blob_flip,0)/face_blob_flip.max()
        # Paso 4 - Aprendizaje Profundo (CNN)
        img_input = img_norm.reshape(1,100,100,3)
        result = model.predict(img_input)
        result = softmax(result)[0]
        confidence_index = result.argmax()
        confidence_score = result[confidence_index]
        label = etiquetas[confidence_index]
        label_text = '{}: {:,.0f}%'.format(label,confidence_score*100)
        #print(label_text)
        color = getcolor(label)
        cv2.rectangle(image,pt1,pt2,color,1)
        cv2.putText(image,label_text,pt1,cv2.FONT_HERSHEY_PLAIN,2,color,2)

cv2.imshow('Prediccion', image)
cv2.waitKey()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 24ms/step


In [7]:
######EN FORMA DE FUNCION#####
def face_mask_prediction(img):
    # Paso 1 - Deteccion de rostros
    image = img.copy()
    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,117,123),swapRB=True)
    #
    face_detection_model.setInput(blob)
    detection = face_detection_model.forward()
    for i in range(0,detection.shape[2]):
        confidence = detection[0,0,i,2]
        if confidence > 0.5:
            box = detection[0,0,i,3:7]*np.array([w,h,w,h])
            box = box.astype(int)
            pt1 = (box[0],box[1])
            pt2 = (box[2],box[3])
            cv2.rectangle(image,pt1,pt2,(0,255,0),1)    
            # Paso 2 - Preprosesamiento de los datos
            face = image[box[1]:box[3],box[0]:box[2]]
            face_blob = cv2.dnn.blobFromImage(face,1,(100,100),(104,117,123),swapRB=True)
            face_blob_squeeze = np.squeeze(face_blob).T
            face_blob_rotate = cv2.rotate(face_blob_squeeze,cv2.ROTATE_90_CLOCKWISE)
            face_blob_flip = cv2.flip(face_blob_rotate,1)
            # Paso 3 - Normalizacion
            img_norm = np.maximum(face_blob_flip,0)/face_blob_flip.max()
            # Paso 4 - Aprendizaje Profundo (CNN)
            img_input = img_norm.reshape(1,100,100,3)
            result = model.predict(img_input)
            result = softmax(result)[0]
            confidence_index = result.argmax()
            confidence_score = result[confidence_index]
            label = etiquetas[confidence_index]
            label_text = '{}: {:,.0f}%'.format(label,confidence_score*100)
            color = getcolor(label)
            cv2.rectangle(image,pt1,pt2,color,1)
            cv2.putText(image,label_text,pt1,cv2.FONT_HERSHEY_PLAIN,2,color,2)
            
    return image

In [17]:
# Comando para reconocimiento
img = cv2.imread('./mascarilla5.jpg')

image = face_mask_prediction(img)

cv2.imshow('Prediccion', image)
cv2.waitKey()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 34ms/step


## Deteccion de mascarillas en tiempo real utilizando las webcam

In [9]:
captura = cv2.VideoCapture(0)

while True:
    ret, frames = captura.read()
    if ret == False:
        break
        
    image = face_mask_prediction(frames)
    cv2.imshow('Deteccion de mascarillas', image)
    if cv2.waitKey(1) == 27:
        break
        
captura.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 23ms/step
